In [ ]:
import os
os.listdir("/content/drive/MyDrive/PRIMERA FINAL")


['model.safetensors',
 'config.json',
 'generation_config.json',
 'special_tokens_map.json',
 'vocab.json',
 'tokenizer.json',
 'training_args.bin',
 'tokenizer_config.json',
 'merges.txt',
 'added_tokens.json']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from transformers import AutoTokenizer

In [ ]:
!pip install -q transformers evaluate rouge-score bert-score


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

drive_model_path = "/content/drive/MyDrive/PRIMERA FINAL"

tokenizer = AutoTokenizer.from_pretrained(drive_model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(drive_model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()


Loading weights:   0%|          | 0/584 [00:01<?, ?it/s]

LEDForConditionalGeneration(
  (led): LEDModel(
    (shared): Embedding(50266, 1024, padding_idx=1)
    (encoder): LEDEncoder(
      (embed_tokens): Embedding(50266, 1024, padding_idx=1)
      (embed_positions): LEDLearnedPositionalEmbedding(4096, 1024)
      (layers): ModuleList(
        (0-11): 12 x LEDEncoderLayer(
          (self_attn): LEDEncoderAttention(
            (longformer_self_attn): LEDEncoderSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (query_global): Linear(in_features=1024, out_features=1024, bias=True)
              (key_global): Linear(in_features=1024, out_features=1024, bias=True)
              (value_global): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (output): Linear(in_features=1024, out_features=1024, bias=True)

In [ ]:
file_to_check = "/content/drive/MyDrive/NewsSumm_perfect_clean.csv"
df=pd.read_csv(file_to_check)

In [ ]:
cluster_sizes = df.groupby('cluster_id').size()

valid_clusters = cluster_sizes[cluster_sizes >= 2].index

df_multi = df[df['cluster_id'].isin(valid_clusters)].reset_index(drop=True)

print("Filtered rows:", len(df_multi))
print("Filtered clusters:", df_multi['cluster_id'].nunique())
print("Avg docs per cluster:",
      df_multi.groupby('cluster_id').size().mean())


Filtered rows: 4335
Filtered clusters: 2060
Avg docs per cluster: 2.104368932038835


In [ ]:
from sklearn.model_selection import train_test_split

clusters = df_multi['cluster_id'].unique()

train_clusters, temp_clusters = train_test_split(
    clusters, test_size=0.2, random_state=42
)

val_clusters, test_clusters = train_test_split(
    temp_clusters, test_size=0.5, random_state=42
)

train_df = df_multi[df_multi['cluster_id'].isin(train_clusters)]
val_df   = df_multi[df_multi['cluster_id'].isin(val_clusters)]
test_df  = df_multi[df_multi['cluster_id'].isin(test_clusters)]

print("Train clusters:", train_df['cluster_id'].nunique())
print("Val clusters:", val_df['cluster_id'].nunique())
print("Test clusters:", test_df['cluster_id'].nunique())


Train clusters: 1648
Val clusters: 206
Test clusters: 206


In [ ]:
def build_cluster_samples(df):
    samples = []
    for cid, group in df.groupby("cluster_id"):
        docs = []

        # sort only if column exists
        if "published_date" in group.columns:
            try:
                group = group.sort_values("published_date")
            except Exception:
                pass

        for _, row in group.iterrows():
            docs.append(f"[DOC]\n{row['article_clean']}")

        samples.append({
            "cluster_id": cid,
            "source": "\n".join(docs),
            "summary": group.iloc[0]['summary_clean']
        })
    return samples


In [ ]:
train_samples = build_cluster_samples(train_df)
val_samples   = build_cluster_samples(val_df)
test_samples  = build_cluster_samples(test_df)

print("Train samples:", len(train_samples))
print("Val samples:", len(val_samples))
print("Test samples:", len(test_samples))


Train samples: 1648
Val samples: 206
Test samples: 206


In [ ]:
test_samples = build_cluster_samples(test_df)


In [ ]:
test_sources = [s["source"] for s in test_samples]
test_references = [s["summary"] for s in test_samples]


In [ ]:
def generate_summaries(texts, max_input_len=512, max_target_len=256):
    predictions = []

    with torch.no_grad():
        for text in texts:
            inputs = tokenizer(
                text,
                max_length=max_input_len,
                truncation=True,
                return_tensors="pt"
            ).to(device)

            summary_ids = model.generate(
                **inputs,
                max_length=max_target_len,
                num_beams=4,
                early_stopping=True
            )

            summary = tokenizer.decode(
                summary_ids[0],
                skip_special_tokens=True
            )
            predictions.append(summary)

    return predictions


predictions = generate_summaries(test_sources)


Input ids are automatically padded from 371 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 337 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 435 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 373 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 359 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 501 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 353 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 401 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 429 to 512 to be a multiple of `config.attention_window`: 512
Input ids are automatically padded from 473 to 512 to be a multiple of `config.att

In [ ]:
import evaluate

rouge = evaluate.load("rouge")

r = rouge.compute(
    predictions=predictions,
    references=test_references
)

rouge_results = {
    "rouge1": r["rouge1"],
    "rouge2": r["rouge2"],
    "rougeL": r["rougeL"]
}

print("📊 ROUGE Scores")
for k, v in rouge_results.items():
    print(f"{k}: {v:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


📊 ROUGE Scores
rouge1: 0.4534
rouge2: 0.2448
rougeL: 0.3330


In [ ]:
def truncate_texts(texts, max_words=300):
    truncated = []
    for t in texts:
        words = t.split()
        truncated.append(" ".join(words[:max_words]))
    return truncated


In [ ]:
preds_trunc = truncate_texts(predictions, max_words=300)
refs_trunc  = truncate_texts(test_references, max_words=300)


In [ ]:
from bert_score import score

P, R, F1 = score(
    preds_trunc,
    refs_trunc,
    model_type="roberta-large",
    lang="en",
    batch_size=8,
    device=device
)

print("BERTScore")
print(f"Precision: {P.mean():.4f}")
print(f"Recall:    {R.mean():.4f}")
print(f"F1:        {F1.mean():.4f}")

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/389 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-large
Key                             | Status     | 
--------------------------------+------------+-
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
pooler.dense.bias               | MISSING    | 
pooler.dense.weight             | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


BERTScore
Precision: 0.8890
Recall:    0.8949
F1:        0.8917
